# 引入需要用到的模型的库包

In [1]:

# GBDT
from sklearn.ensemble import GradientBoostingRegressor
# XGBoost
import xgboost as xgb
# LightGBM
import lightgbm as lgb
# CatBoost
import catboost as cb

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import pickle
import multiprocessing
from sklearn.preprocessing import StandardScaler
ss = StandardScaler() 
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,LinearRegression,LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

I:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# 读取文件

## 读取特征




### pinkman 

In [2]:

features_path = 'D:/城市-房产租金预测/stacking/pinkman.pkl'
data_fp = open(features_path, 'rb')
train, test = pickle.load(data_fp)
print("读取pinkman特征")
data_fp.close()
target = train.pop("tradeMoney")
test.drop(["ID"],axis = 1, inplace=True)
train.drop(["ID"],axis = 1, inplace=True)
# X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.4)
pinkman_train, pinkman_test, pinkman_target = train,test,target
pinkman_train = pinkman_train.astype(float)
pinkman_test = pinkman_test.astype(float)
print(pinkman_train.shape,pinkman_test.shape)

读取pinkman特征
(40006, 243) (2401, 243)


In [3]:
pinkman_test[:5]

,Bath,Hall,Room,area,buildYear,houseDecoration,houseFloor,houseToward,plate,pv,...,cluster10,communityName_houseToward_nunique,communityName_houseType_nunique,plate_houseToward_nunique,plate_houseType_nunique,communityName_totalFloor_mean,plate_totalFloor_mean,communityName_houseDecoration_mean,plate_houseDecoration_mean,pre
40006,1.0,1.0,2.0,76.20,2006.0,0.0,0.0,5.0,50.0,15244.0,...,2.0,3.0,3.0,7.0,23.0,8.846154,9.866115,1.076923,0.932367,5090.613296
40007,1.0,1.0,1.0,35.00,1995.0,0.0,2.0,4.0,28.0,687.0,...,3.0,2.0,2.0,8.0,14.0,6.000000,13.137778,0.000000,0.546667,3868.608230
40008,1.0,2.0,2.0,93.00,2003.0,0.0,1.0,5.0,53.0,33164.0,...,2.0,2.0,3.0,9.0,22.0,11.000000,10.500305,0.600000,0.685400,6574.183442
40009,1.0,1.0,2.0,70.00,1997.0,0.0,1.0,5.0,53.0,42179.0,...,2.0,2.0,2.0,9.0,22.0,6.500000,10.500305,0.000000,0.685400,4429.405550
40010,1.0,2.0,3.0,103.75,2013.0,0.0,1.0,6.0,53.0,17471.0,...,2.0,6.0,6.0,9.0,22.0,18.000000,10.500305,0.984127,0.685400,9042.396010


### Jason特征V1

In [4]:
path = 'D:/城市-房产租金预测/stacking/'
train_df = open(path+"Jason_stacking_train_v1.csv","rb")
test_df = open(path+"Jason_stacking_test_v1.csv","rb")
Jason_train_v1 = pd.read_csv(train_df)
Jason_test_v1 = pd.read_csv(test_df)
Jason_target_v1 = Jason_train_v1.pop("tradeMoney")
categorical_feats = ['houseDecoration', 'plate', 'rentType']
def cat2num(data):
    for i in categorical_feats:
        data[i] = data[i].map(dict(zip(data[i].unique(), range(0, data[i].nunique()))))
    return data
Jason_train_v1 = cat2num(Jason_train_v1)
Jason_test_v1 = cat2num(Jason_test_v1)
Jason_train_v1.drop(["ID"],axis = 1, inplace=True)
Jason_test_v1.drop(["ID"],axis = 1, inplace=True)
print(Jason_train_v1.shape,Jason_test_v1.shape)



(39618, 43) (2401, 43)


### Jason特征V2

In [5]:
path = 'D:/城市-房产租金预测/stacking/'
train_df = open(path+"Jason_stacking_train_v2.csv","rb")
test_df = open(path+"Jason_stacking_test_v2.csv","rb")
Jason_train_v2 = pd.read_csv(train_df)
Jason_test_v2 = pd.read_csv(test_df)
Jason_target_v2 = Jason_train_v2.pop("tradeMoney")
categorical_feats = ['houseDecoration', 'plate', 'rentType']
def cat2num(data):
    for i in categorical_feats:
        data[i] = data[i].map(dict(zip(data[i].unique(), range(0, data[i].nunique()))))
    return data
Jason_train_v2 = cat2num(Jason_train_v2)
Jason_test_v2 = cat2num(Jason_test_v2)
Jason_train_v2.drop(["ID"],axis = 1, inplace=True)
Jason_test_v2.drop(["ID"],axis = 1, inplace=True)
print(Jason_train_v2.shape,Jason_test_v2.shape)



(39618, 42) (2401, 42)


In [6]:
Jason_train_v2[:5]

,area,totalFloor,houseDecoration,plate,buildYear,bedroom,hall,wc,tradeMonth,communityNameW0,...,Groupby_communityName_pv_std,Groupby_communityName_bedroom_std,Groupby_communityName_totalTradeMoney_median,Groupby_communityName_totalFloor_mean,Count_communityName,Count_totalFloor,Count_communityName_newWorkers,Count_communityName_bedroom,rentType,train_pre
0,68.06,16,0,0,1953,2,1,1,11,0.038550,...,3505.978988,0.516398,5.077100e+08,10.166667,6,1197,1,2,0,5971.670451
1,125.55,14,1,1,2007,3,2,2,12,-0.047079,...,2142.516604,0.522233,5.000000e+06,13.181818,11,2275,1,6,0,2419.196923
2,132.00,32,0,2,1994,3,2,2,12,-0.035118,...,5048.220426,0.762001,1.687285e+09,29.451613,31,239,4,12,0,12846.002864
3,57.00,17,2,3,1994,1,1,1,12,-0.023838,...,13826.332598,0.389118,4.758300e+08,17.833333,60,1441,3,53,0,2035.895644
4,129.00,2,3,4,1994,3,2,3,11,0.023886,...,7521.251340,0.619139,1.093100e+08,4.750000,16,194,2,8,0,3189.746127


### hero

In [7]:
path = 'D:/城市-房产租金预测/stacking/'
train_df = open(path+"hero_stacking_train.csv","rb")
test_df = open(path+"hero_stacking_test.csv","rb")
hero_train = pd.read_csv(train_df)
hero_test = pd.read_csv(test_df)
# hero_train.drop(["ID"],axis = 1, inplace=True)
# hero_test.drop(["ID"],axis = 1, inplace=True)
# hero_train["ID"] = hero_train["ID"].astype(int)
# hero_test["ID"] = hero_test["ID"].astype(int)
hero_train.drop(["Unnamed: 0","ID","city"],axis = 1, inplace=True)
hero_test.drop(["Unnamed: 0","ID","city"],axis = 1, inplace=True)
print(hero_train.shape,hero_test.shape)

(39504, 151) (2401, 151)


In [8]:
hero_train[:4]

,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,region,plate,...,plateW0,plateW1,plateW2,plateW3,plateW4,plateW5,plateW6,plateW7,cluster1,train_pre
0,68.06,0,0,0,16,0,0,51,1,64,...,0.550193,-2.172251,-1.498352,0.640507,1.451142,0.468397,1.712500,-2.903254,7,5865.904441
1,125.55,1,1,1,14,0,1,130,2,49,...,0.220966,-3.330363,0.561736,1.946966,2.904222,0.168849,0.381790,-2.022188,2,1790.163705
2,57.00,1,2,1,17,0,2,313,2,51,...,-0.186082,-2.562744,-0.801996,-1.299095,3.000661,0.656332,-0.292044,0.742765,2,1946.767365
3,129.00,1,3,0,2,0,3,1257,3,44,...,0.348185,-0.729719,0.447798,0.350268,2.359439,0.482003,0.714489,-1.120475,9,3568.743583


### heitao

In [9]:
features_path = 'D:/城市-房产租金预测/stacking/heitao.pkl'
data_fp = open(features_path, 'rb')
heitao_train, heitao_test = pickle.load(data_fp)
heitao_target = heitao_train.pop("target")
heitao_train.drop(["ID"],axis = 1, inplace=True)
heitao_test.drop(["ID"],axis = 1, inplace=True)
print("读取特征")
data_fp.close()
print(heitao_train.shape,heitao_test.shape)


读取特征
(40126, 232) (2401, 232)


In [10]:
heitao_train[:4]

,Bath,Hall,Room,Room_Bath,Schoo_ratio,all_SchoolNum,all_SchoolNum_buildYear_cluster_mean,all_SchoolNum_communityName_cluster_mean,all_SchoolNum_houseDecoration_cluster_mean,all_SchoolNum_plate_cluster_mean,...,trainsportNum_all_hospitalNum_cluster5_mean,trainsportNum_all_mall_cluster5_mean,trainsportNum_otherNum_cluster5_mean,all_SchoolNum_all_hospitalNum_cluster5_mean,all_SchoolNum_all_mall_cluster5_mean,all_SchoolNum_otherNum_cluster5_mean,all_hospitalNum_all_mall_cluster5_mean,all_hospitalNum_otherNum_cluster5_mean,all_mall_otherNum_cluster5_mean,pre
0,1,1,2,0.666667,0.010,0.737603,2.637199,0.734387,3.928964,0.736073,...,0.668591,0.743583,2.248453,0.668591,0.743583,2.248453,0.743583,2.248453,2.248453,3548.599153
1,2,2,3,0.750000,0.050,0.537363,3.287083,0.536032,4.087380,0.536811,...,0.926685,1.250710,2.533498,0.926685,1.250710,2.533498,1.250710,2.533498,2.533498,1904.003447
2,2,2,3,0.750000,0.026,2.555343,3.998208,2.564502,3.928964,2.561292,...,1.799298,0.999422,3.515890,1.799298,0.999422,3.515890,0.999422,3.515890,3.515890,15601.905397
3,1,1,1,1.000000,0.047,4.096610,3.998208,4.099168,3.048312,4.101421,...,5.125046,1.727672,3.852481,5.125046,1.727672,3.852481,1.727672,3.852481,3.852481,1857.312781


### yb

In [11]:
features_path = 'D:/城市-房产租金预测/stacking/yb.pkl'
data_fp = open(features_path, 'rb')
yb_train, yb_test = pickle.load(data_fp)
yb_target = yb_train.pop("tradeMoney")
yb_train.drop(["ID"],axis = 1, inplace=True)
yb_test.drop(["ID"],axis = 1, inplace=True)
print("读取特征")
data_fp.close()
print(yb_train.shape,yb_test.shape,yb_target.shape)


读取特征
(40006, 242) (2401, 242) (40006,)


### airen

In [12]:
features_path = 'D:/城市-房产租金预测/stacking/airen.pkl'
data_fp = open(features_path, 'rb')
airen_train, airen_test = pickle.load(data_fp)
airen_target = airen_train.pop("tradeMoney")
airen_train.drop(["ID","areaMoney"],axis = 1, inplace=True)
airen_test.drop(["ID"],axis = 1, inplace=True)
print("读取特征")
data_fp.close()
print(airen_train.shape,airen_test.shape,airen_target.shape)

读取特征
(37663, 521) (4870, 521) (37663,)


# 结果简要分析

In [4]:
from sklearn.metrics import r2_score
def online_score(pred):
    print("预测结果最大值：{},预测结果最小值：{}".format(pred.max(),pred.min()))
    # a榜测分
    conmbine1 = pd.read_csv("D:/城市-房产租金预测/best_result/sub_a_913.csv",engine = "python",header=None)
    score1 = r2_score(pred, conmbine1)
    # b榜测分
    conmbine3 = pd.read_csv("D:/城市-房产租金预测/best_result/sub_b_9194.csv",engine = "python",header=None)
    score3= r2_score(pred, conmbine3)

    print("对比919分数:{}".format(score3))
    

# 定义模型参数

In [14]:
#LASSO Regression :  该模型可能对异常值非常敏感。 所以我们需要让它们更加健壮。 为此，我们在管道上使用sklearn的Robustscaler（）方法
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
 
#Elastic Net Regression 同上
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
 
#Gradient Boosting Regression :huber损失有很好的鲁棒性
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
 
#xgb
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=5000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                              nthread = -1)


#lgb
model_lgb = lgb.LGBMRegressor(objective='regression',
                              num_leaves=31,
                              boosting = "gbdt",
                              learning_rate=0.01, 
                              n_estimators=720,
                              max_bin = 55, 
                              bagging_fraction = 0.8,
                              bagging_freq = 1, 
                              feature_fraction = 0.85,
                              feature_fraction_seed=9, 
                              bagging_seed=23,
                              min_child_samples = 20,
                              min_data_in_leaf =20, 
                              min_sum_hessian_in_leaf = 11,
                              metric = 'rmse',
                              lambda_l1 = 0.3,
                              nthread = 4,
                              )
model_cb = cb.CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.5,eval_metric = "R2",leaf_estimation_method = "Newton",
                            l2_leaf_reg=3
                            )

# Stacking

 每个人的特征 Stacking 
     
     数据输入第一层模型，输出即将喂给第二层模型特征
     
 6个Stacking的结果取平均

In [1]:
train[3]

NameError: name 'train' is not defined

## 第一层模型(5折交叉验证取平均)

In [18]:
### 5折stacking
def stack(X_train,X_test,y_train):
    clfs1 = [ GBoost,model_xgb,model_lgb]
    X_train_stack  = np.zeros((X_train.shape[0], len(clfs1)))
    X_test_stack = np.zeros((X_test.shape[0], len(clfs1))) 
    folds = KFold(n_splits=5, shuffle=True, random_state=2333)
    # skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=1)
    for i,clf in enumerate(clfs1):
        print("分类器：{}".format(clf))
        X_stack_test_n = np.zeros((X_test.shape[0], 5))
        for j,(trn_idx, val_idx)in enumerate(folds.split(X_train,y_train)):
            tr_x = X_train.iloc[trn_idx]
            tr_y = y_train.iloc[trn_idx]
            clf.fit(tr_x, tr_y)
            #生成stacking训练数据集
            X_train_stack[val_idx, i] = clf.predict(X_train.iloc[val_idx])
            X_stack_test_n[:,j] = clf.predict(X_test)

        #生成stacking测试数据集
        X_test_stack[:,i] = X_stack_test_n.mean(axis=1) 

    X_train_stack1 = pd.DataFrame(X_train_stack)
    X_test_stack1 = pd.DataFrame(X_test_stack)
    X_train_stack1.shape,X_test_stack1.shape
    
    X_train_stack1 = X_train_stack1.reset_index(drop=True)
    X_train = X_train.reset_index(drop=True)
    X_test_stack1 = X_test_stack1.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
   ##  第一层模型的输出特征合并
    train = pd.concat([X_train_stack1,X_train],axis = 1)
    test = pd.concat([X_test_stack1,X_test],axis = 1)
    print(train.shape,test.shape)
    return train,test


## 第二层模型(LightGBM)

In [19]:
def layer2(train,test,target):
    params = {
        'num_leaves': 31,
        'min_data_in_leaf': 20,
        'min_child_samples':20,
        'objective': 'regression',
        'learning_rate': 0.01,
        "boosting": "gbdt",
        "feature_fraction": 0.8,
        "bagging_freq": 1,
        "bagging_fraction": 0.85,
        "bagging_seed": 23,
        "metric": 'rmse',
        "lambda_l1": 0.3,
        "nthread": 4,
    }
    # train,test,target = train[best_fea],test[best_fea],pd.DataFrame(target)
    # train,test = X_train_stack,X_test_stack
    folds = KFold(n_splits=5, shuffle=True, random_state=2333)

    oof_lgb = np.zeros(len(train))
    predictions_lgb = np.zeros(len(test))
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold {}".format(fold_))
    #     trn_data = lgb.Dataset(train.iloc[trn_idx], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
    #     val_data = lgb.Dataset(train.iloc[val_idx], label=target.iloc[val_idx], categorical_feature=categorical_feats)
        trn_data = lgb.Dataset(train.iloc[trn_idx], label=target.iloc[trn_idx])
        val_data = lgb.Dataset(train.iloc[val_idx], label=target.iloc[val_idx])

        num_round = 10000
        clf = lgb.train(params, trn_data, num_round, valid_sets=[trn_data, val_data], verbose_eval=500,
                        early_stopping_rounds=200)

        oof_lgb[val_idx] = clf.predict(train.iloc[val_idx], num_iteration=clf.best_iteration)
#         OOF_lgb  = pd.DataFrame()
#     #     OOF_lgb[fold_] = clf.predict(train, num_iteration=clf.best_iteration)



#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["feature"] = train.columns
#         fold_importance_df["importance"] = clf.feature_importance()
#         fold_importance_df["fold"] = fold_ + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions_lgb += clf.predict(test, num_iteration=clf.best_iteration) / folds.n_splits

    print("CV Score: {:<8.5f}".format(r2_score(target, oof_lgb)))
    import datetime
    def create_submission(prediction, score,model_name):
        now = datetime.datetime.now()
        sub_file = 'D:/城市-房产租金预测/result/submission_' +model_name+ str(score) + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
        print ('Creating submission: ', sub_file)
    #     print(predictions_lgb.min(),predictions_lgb.max())
        pd.DataFrame({ 'trademoney': prediction}).to_csv(sub_file, index=False,header=None)

    model_name='lgbm'
    score=r2_score(target, oof_lgb)
    create_submission(predictions_lgb, score,model_name)

#     print(online_score(predictions_lgb))
    return predictions_lgb


## 训练

In [20]:
pinkman_train,pinkman_test = stack(pinkman_train,pinkman_test,pinkman_target)
pinkman_pre = layer2(pinkman_train,pinkman_test,pinkman_target)



分类器：GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.05, loss='huber', max_depth=4,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=15, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='auto', random_state=5,
                          subsample=1.0, tol=0.0001, validation_fraction=0.1,
                          verbose=0, warm_start=False)
分类器：XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=0.4603, gamma=0.0468, importance_type='gain',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1.7817, missing=None, n_estimators=5000, n_jobs=1,
             nthread=-1, objective='reg:linear', ran

In [21]:
Jason_train_v1.rename(columns={'train_pre':'pre'}, inplace = True)
Jason_test_v1.rename(columns={'test_pre':'pre'}, inplace = True)

Jason_train_v2.rename(columns={'train_pre':'pre'}, inplace = True)
Jason_test_v2.rename(columns={'test_pre':'pre'}, inplace = True)

Jason_train_v1,Jason_test_v1 = stack(Jason_train_v1,Jason_test_v1,Jason_target_v1)
Jason_pre_v1 = layer2(Jason_train_v1,Jason_test_v1,Jason_target_v1)

Jason_train_v2,Jason_test_v2 = stack(Jason_train_v2,Jason_test_v2,Jason_target_v2)
Jason_pre_v2 = layer2(Jason_train_v2,Jason_test_v2,Jason_target_v2)



分类器：GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.05, loss='huber', max_depth=4,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=15, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='auto', random_state=5,
                          subsample=1.0, tol=0.0001, validation_fraction=0.1,
                          verbose=0, warm_start=False)
分类器：XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=0.4603, gamma=0.0468, importance_type='gain',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1.7817, missing=None, n_estimators=5000, n_jobs=1,
             nthread=-1, objective='reg:linear', ran

In [22]:
heitao_train,heitao_test = stack(heitao_train,heitao_test,heitao_target)
heitao_pre = layer2(heitao_train,heitao_test,heitao_target)


分类器：GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.05, loss='huber', max_depth=4,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=15, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='auto', random_state=5,
                          subsample=1.0, tol=0.0001, validation_fraction=0.1,
                          verbose=0, warm_start=False)
分类器：XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=0.4603, gamma=0.0468, importance_type='gain',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1.7817, missing=None, n_estimators=5000, n_jobs=1,
             nthread=-1, objective='reg:linear', ran

In [23]:
airen_train,airen_test = stack(airen_train,airen_test,airen_target)
airen_pre = layer2(airen_train,airen_test,airen_target)
# 由于爱人的使用了test_a的数据 所以需要切分一下
airen_pre = airen_pre[:len(heitao_pre)]

分类器：GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.05, loss='huber', max_depth=4,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=15, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='auto', random_state=5,
                          subsample=1.0, tol=0.0001, validation_fraction=0.1,
                          verbose=0, warm_start=False)
分类器：XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=0.4603, gamma=0.0468, importance_type='gain',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1.7817, missing=None, n_estimators=5000, n_jobs=1,
             nthread=-1, objective='reg:linear', ran

In [26]:
yb_train,yb_test = stack(yb_train,yb_test,yb_target)
yb_pre = layer2(yb_train,yb_test,yb_target)

分类器：GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.05, loss='huber', max_depth=4,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=15, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='auto', random_state=5,
                          subsample=1.0, tol=0.0001, validation_fraction=0.1,
                          verbose=0, warm_start=False)
分类器：XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=0.4603, gamma=0.0468, importance_type='gain',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1.7817, missing=None, n_estimators=5000, n_jobs=1,
             nthread=-1, objective='reg:linear', ran

In [44]:
pre = (pinkman_pre + Jason_pre_v1 + Jason_pre_v2 + airen_pre+ yb_pre+ heitao_pre)/6
online_score(pre)

预测结果最大值：15118.27064880905,预测结果最小值：1355.7491137830068
对比913分数:0.9930013663744774
对比918分数:0.997513390896551
对比919分数:0.9985484433504949


In [45]:
pd.DataFrame(pre).to_csv("0.9985484433504949.csv",header=None,index=None)

In [50]:
online_score(heitao_pre)

预测结果最大值：15606.824182987737,预测结果最小值：1222.1235852723944
对比913分数:0.980332815979435
对比918分数:0.9895456161867269
对比919分数:0.9921747694981579


In [ ]:
log1p_y_train = np.log1p(target)
ENet = ElasticNet(alpha=1e-9, l1_ratio=.59, random_state=5)
# 采用平滑后的y_train进行模型训练
ENet.fit(train, log1p_y_train)
# 对训练集预测
pred_log1p = ENet.predict(train)
# 模型评分
score = r2_score(log1p_y_train,pred_log1p)
print(score)
# 对测试集预测
ENet_pred = ENet.predict(test)
# 将测试集结果去掉平滑
ENet_pred = np.expm1(ENet_pred)
# 保存文件
pd.DataFrame(ENet_pred).to_csv("ENet_{}.csv".format(score),header=None,index=None)
online_score(ENet_pred)

In [ ]:
log1p_y_train = np.log1p(target)
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=1e-9, l1_ratio=.59, random_state=5))
# 采用平滑后的y_train进行模型训练
ENet.fit(train, log1p_y_train)
# 对训练集预测
pred_log1p = ENet.predict(train)
# 模型评分
score = r2_score(log1p_y_train,pred_log1p)
print(score)
# 对测试集预测
ENet_pred = ENet.predict(test)
# 将测试集结果去掉平滑
ENet_pred = np.expm1(ENet_pred)
# 保存文件
pd.DataFrame(ENet_pred).to_csv("ENet_{}.csv".format(score),header=None,index=None)
online_score(ENet_pred)

In [ ]:
log1p_y_train = np.log1p(target)
lasso = make_pipeline(RobustScaler(), Lasso(alpha =1e-9, random_state=5))

# 采用平滑后的y_train进行模型训练
lasso.fit(train, log1p_y_train)
# 对训练集预测
pred_log1p = lasso.predict(train)
# 模型评分
score = r2_score(log1p_y_train,pred_log1p)
print(score)
# 对测试集预测
pred = lasso.predict(test)
# 将测试集结果去掉平滑
lasso_pred = np.expm1(pred)
# 保存结果
pd.DataFrame(lasso_pred).to_csv("lasso_{}.csv".format(score),header=None,index=None)
print(online_score(lasso_pred))

In [ ]:
predictions = (predictions_lgb * 0.9 +  GBoost_pred * 0.5 + xgb_pred * 0.5)/(0.9+0.5+0.5)

In [ ]:
print(online_score(predictions))

# ID处理(抛弃)

In [ ]:
heitao_ID = np.array(heitao_train["ID"])
pinkman_ID = np.array(pinkman_train["ID"])
Jason_ID_v1 = np.array(Jason_train_v1["ID"])
Jason_ID_v2 = np.array(Jason_train_v2["ID"])
hero_ID = np.array(hero_train["ID"])
print(len(heitao_ID),len(pinkman_ID))
ID = set(heitao_ID) & set(pinkman_ID) & set(Jason_ID_v1) & set(Jason_ID_v2)  & set(hero_ID)
print(len(ID))
# ID = pd.DataFrame(ID)

In [ ]:
def drop_for_ID(train,test):
    drop_ID = set(train.ID).difference(ID)
    print("原始数据集大小{},将要删除{}个ID,正在删除...".format(train.shape,len(drop_ID)))
    for i in drop_ID:
        train.drop(train[(train['ID'] == i)].index, inplace=True)
#         test.drop(test[(test['ID'] == i)].index, inplace=True)
    print("已删除ID,正在根据ID排序...")
    train.sort_values(by="ID")
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    print("根据ID排序完成！！！")
    print("剩余训练集大小{}...".format(train.shape))
    return train,test

In [ ]:
pinkman_train,pinkman_test = drop_for_ID(pinkman_train,pinkman_test)
# pinkman_train["ID"]

heitao_train,heitao_test = drop_for_ID(heitao_train,heitao_test)
# heitao_train["ID"]

Jason_train_v1,Jason_test_v1 = drop_for_ID(Jason_train_v1,Jason_test_v1)
# Jason_train_v1["ID"]

Jason_train_v2,Jason_test_v2 = drop_for_ID(Jason_train_v2,Jason_test_v2)
# Jason_train_v2["ID"]

hero_train,hero_test = drop_for_ID(hero_train,hero_test)
# hero_train["ID"]

In [ ]:
# ID处理

heitao_ID = np.array(heitao_train["ID"])
pinkman_ID = np.array(pinkman_train["ID"])
Jason_ID_v1 = np.array(Jason_train_v1["ID"])
Jason_ID_v2 = np.array(Jason_train_v2["ID"])
hero_ID = np.array(hero_train["ID"])
print(len(heitao_ID),len(pinkman_ID))
ID = set(heitao_ID) & set(pinkman_ID) & set(Jason_ID_v1) & set(Jason_ID_v2)  & set(hero_ID)
print(len(ID))
# ID = pd.DataFrame(ID)

def drop_for_ID(train,test):
    drop_ID = set(train.ID).difference(ID)
    print("原始数据集大小{},将要删除{}个ID,正在删除...".format(train.shape,len(drop_ID)))
    for i in drop_ID:
        train.drop(train[(train['ID'] == i)].index, inplace=True)
#         test.drop(test[(test['ID'] == i)].index, inplace=True)
    print("已删除ID,正在根据ID排序...")
    train.sort_values(by="ID")
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    print("根据ID排序完成！！！")
    print("剩余训练集大小{}...".format(train.shape))
    return train,test

pinkman_train,pinkman_test = drop_for_ID(pinkman_train,pinkman_test)
# pinkman_train["ID"]

heitao_train,heitao_test = drop_for_ID(heitao_train,heitao_test)
# heitao_train["ID"]

Jason_train_v1,Jason_test_v1 = drop_for_ID(Jason_train_v1,Jason_test_v1)
# Jason_train_v1["ID"]

Jason_train_v2,Jason_test_v2 = drop_for_ID(Jason_train_v2,Jason_test_v2)
# Jason_train_v2["ID"]

hero_train,hero_test = drop_for_ID(hero_train,hero_test)
# hero_train["ID"]

In [5]:
pre= pd.read_csv("D:/AIfuture/CODE1(如果可以，麻烦使用这一版，之前那版在整理时出现了bug，但是结果都一样)/data/submit.csv",engine = "python",header = None)

online_score(pre)

预测结果最大值：0    15147.922324
dtype: float64,预测结果最小值：0    1332.325739
dtype: float64
对比913分数:0.9932508501940013
对比918分数:0.9980113048560919
对比919分数:0.9998189065713886
